# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**:

**ID**:

> **Due Date**
>
> Wednesday, 10/16/24, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [96]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Local/Coding Projects/BEE4570/lab-3`


In [97]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

`JuMP` requires the loading of a solver. \[Each supported solver works
for certain classes of problems, and some are open source while others
require a commercial license\]. We will use the `HiGHS` solver, which is
open source and works for linear, mixed integer linear, and quadratic
programs.

In this lab we will walk through the steps involved in coding a linear
program in HiGHS, solving it, and querying the solution.

## Exercise (3 points)

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
$\$0.12$ per bf to purchase and fir costs $\$0.08$ per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board, which yields no
revenue for the mill.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.

First, we need to identify our decision variables. While there are
several options, we will use $G_i$, the amount of each grade the mill
produces (in \$/1000 bf).

Using these decision variables, formulate a linear program to maximize
the profit of the mill subject to the supply constraints on spruce and
fir.

> **JuMP Syntax**
>
> The core pieces of setting up a `JuMP` model involve specifying the
> model and adding variables, the objective, and constraints. At the
> most simple level, this syntax looks like this:
>
> ``` julia
> m = Model(HiGHS.Optimizer)
> @variable(m, lb <= x <= ub) # if you do not have upper or lower bounds, you can drop those accordingly
> @variable(m, lb <= y <= ub)
> @objective(m, Max, 100x + 250y) # replace Max with Min depending on the problem
> @constraint(m, label, 6x + 12y <= 80) # replace "label" with some meaningful string you would like to use later to query shadow prices, or drop it
> ```
>
> You can add more constraints or more variables as needed.

> **Using Array Syntax**
>
> You can set up multiple variables or constraints at once using array
> syntax. For example, the following are equivalent:
>
> ``` julia
> @variable(m, G1 >= 0)
> @variable(m, G2 >= 0)
> @variable(m, G3 >= 0)
> ```
>
> and
>
> ``` julia
> @variable(m, G[1:3] >= 0)
> ```
>
> You can also set up multiple constraints using arrays of coefficients
> and/or bounds. For example:
>
> ``` julia
> I = 1:3
> d = [0; 3; 5]
> @constraint(m, demand[i in I], G[i] >= d[i])
> ```

`JuMP` is finicky about changing objects and constraints, so I recommend
setting up all of the model syntax in one notebook cell, which is what
we will do here.

## Solution ##

First, we should write out the problem formulation including the objective function and constraints. The objective function should represent the total profit from the different grades of plywood, so it should consider not only the revenue per 1000 bf of plywood, but also the costs of sourcing the amount of spruce and fir needed. I decided th turn everything into units of 1 bf, instead of 1000, to make things a little clearer. The variables $x_1, x_2, x_3$ in the math below represent the amount of plywood grades 1, 2, and 3 respectively.

$ \max \text{ }  (0.4 - 0.5 \times 0.12 - 1.5 \times 0.08) x_1 + (0.52 - 0.12 -2\times 0.08) x_2 + (0.7 +1.5\times 0.12 + 2\times 0.08) x_3 $

Subject to:

$0.5 x_1 + x_2 + 1.5 x_1 \leq 320000$

$1.5 x_1 + 2 x_2 + 2 x_3 \leq 720000$

$ -x_1, -x_2, -x_3 \leq 0$

After encoding this model into the code below, finding the optimal solution is as easy as running the code blocks that solve the model. The results indicate that 352000 bf of grade 1 plywood should be made, as well as 96000 bf of the grade 3 wood. No plywood of the 2nd grade should be made in this optimal solution. The profit from this, the value of the objective function, is $112,000

In [105]:
forest_model = Model(HiGHS.Optimizer) # initialize model object

# Variables and non negativity constraints
@variable(forest_model, G[1:3] >= 0) # non-negativity constraints

# uncomment the following lines and add the objective
@objective(forest_model, Max, .400*G[1] - 0.5*.120*G[1] - 1.5*.080*G[1]
                             + .520*G[2] - 1*.120*G[2] - 2*.080*G[2]
                             + .700*G[3] - 1.5*.120*G[3] - 2*.080*G[3])

# constraints
@constraint(forest_model, s_max, 0.5*G[1] + G[2] + 1.5*G[3] <= 320000)    # amount of spruce constraint
@constraint(forest_model, f_max, 1.5*G[1] + 2*G[2] + 2*G[3] <= 720000)    # amount of fir constraint

print(forest_model) # this outputs a nicely formatted summary of the model so you can check your specification

Max 0.22000000000000003 G[1] + 0.24000000000000002 G[2] + 0.36 G[3]
Subject to
 s_max : 0.5 G[1] + G[2] + 1.5 G[3] ≤ 320000
 f_max : 1.5 G[1] + 2 G[2] + 2 G[3] ≤ 720000
 G[1] ≥ 0
 G[2] ≥ 0
 G[3] ≥ 0


Next, to optimize, use the `optimize!()` function:

In [99]:
optimize!(forest_model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-01, 2e+00]
  Cost   [2e-01, 4e-01]
  Bound  [0e+00, 0e+00]
  RHS    [3e+05, 7e+05]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 3(-6); elements 6(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.1999884144e-01 Ph1: 2(8.5); Du: 3(0.819999) 0s
          2    -1.1200000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1200000000e+05
HiGHS run time      :          0.00


You should get confirmation that a solution was found; if one was not,
there’s a chance something was wrong with your model formulation.

To find the values of the decision variables, use `value()` (which can
be broadcasted over variable arrays):

In [100]:
@show value.(G);

value.(G) = [352000.00000000006, 0.0, 95999.99999999999]


Similarly, `objective_value()` finds the optimal value of the objective:

In [101]:
@show objective_value(forest_model);

objective_value(forest_model) = 112000.00000000001


Finally, we can find the dual values of the constraints with
`shadow_price()`. Do this for the constraints in your model using the
block below.

In [104]:
@show shadow_price(G1_input);
@show shadow_price(G2_input);
@show shadow_price(G3_input);

shadow_price(G1_input) = 0.4
shadow_price(G2_input) = 0.52
shadow_price(G3_input) = 0.7


`JuMP` also lets you evaluate other expressions that you might be
interested in based on the solutions. For example, you can use the
following block to calculate the total amount of plywood the mill would
produce under the optimal solution:

In [103]:
@expression(forest_model, total_plywood, sum(G))
@show value.(total_plywood);

value.(total_plywood) = 448000.00000000006


## References

Put any consulted sources here, including classmates you worked with/who
helped you.